<a href="https://colab.research.google.com/github/arjuns238/MachineTranslation/blob/main/MachineTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from torch.utils.data import Dataset, DataLoader,random_split
from typing import Iterable, List
import torchtext
from sklearn.model_selection import train_test_split
import math

# Place-holders
token_transform = {}
vocab_transform = {}

In [ ]:
# # Installing dependencies
# !pip install -U torchdata
# !pip install -U spacy
!pip install 'portalocker>=2.0.0'
# !python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 33.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
batch_size = 64
block_size = 128
learning_rate = 1e-2
max_iters = 5000
eval_interval = 500
eval_iters = 200
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_embd = 384
dropout = 0.2
no_of_heads = 6
n_layer = 6
device
SRC_LANGUAGE = 'Fr'
TGT_LANGUAGE = 'En'
device

'cpu'

In [ ]:
data = pd.read_csv("/content/eng-fra.txt", sep="\t", header=None)
data = data.set_axis(['En','Fr'], axis = 1) # Rename indices
data.tail()

,En,Fr
135837,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
135838,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
135839,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
135840,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
135841,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [ ]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='fr_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter[language]:
        yield token_transform[language](data_sample)

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    print(ln)
    # Training data Iterator
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(data, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

Fr
En


In [ ]:
print(vocab_transform["En"].lookup_token(200))
print(vocab_transform["En"].lookup_indices(["left"]))
vocab_size_src = len(vocab_transform[SRC_LANGUAGE])
vocab_size_tgt = len(vocab_transform[TGT_LANGUAGE])
print(f"Vocab size for {SRC_LANGUAGE} = {vocab_size_src}")
print(f"Vocab size for {TGT_LANGUAGE} = {vocab_size_tgt}")

left
[200]
Vocab size for Fr = 24554
Vocab size for En = 14875


In [ ]:
# Step 1: Encode a sentence
def encode_sentence(sentence: str, language: str, tokenizer, vocab) -> List[int]:
    # Tokenize the sentence
    tokens = tokenizer(sentence)
    # Convert tokens to indices using vocabulary
    indices = vocab_transform[language].lookup_indices(tokens)
    return indices

# Step 2: Decode a sequence
def decode_sequence(indices: List[int], language: str, vocab) -> str:
    # Convert indices to tokens
    tokens = [vocab_transform[language].lookup_token(index) for index in indices]
    # Remove <bos> and <eos> tokens if present
    if tokens[0] == '<bos>':
        tokens = tokens[1:]
    if tokens[-1] == '<eos>':
        tokens = tokens[:-1]
    # Convert tokens to a sentence
    sentence = ""
    for token in tokens:
        if token == '<bos>' or  token == '<eos>' or token == '<pad>':
            continue
        sentence = sentence + " " + token
    return sentence

# Example usage
sentence = "Je suis froid"
encoded = encode_sentence(sentence, SRC_LANGUAGE, token_transform[SRC_LANGUAGE], vocab_transform[SRC_LANGUAGE])
decoded = decode_sequence(encoded, SRC_LANGUAGE, vocab_transform[SRC_LANGUAGE])
print("Original sentence:", sentence)
print("Encoded sequence:", encoded)
print("Decoded sentence:", decoded)

Original sentence: Je suis froid
Encoded sequence: [6, 34, 448]
Decoded sentence:  Je suis froid


In [ ]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_sample = text_transform[SRC_LANGUAGE](src_sample.rstrip("\n"))
        src_sample = src_sample[:block_size]
        tgt_sample = text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n"))
        tgt_sample = tgt_sample[:block_size]
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)

        # src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        # tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)

    return src_batch.T, tgt_batch.T

In [ ]:
sen = data["Fr"][0]
sen = data["En"][0]

print(sen)
outputText = text_transform[TGT_LANGUAGE](sen.rstrip("\n"))
engText = text_transform[SRC_LANGUAGE](sen.rstrip("\n"))

print(outputText)
print(engText)

Go.
tensor([  2, 572,   4,   3])
tensor([2, 0, 4, 3])


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, inputText, outputText):
        self.inputText = inputText
        self.outputText = outputText

    def __len__(self):
        return len(self.inputText)

    def __getitem__(self, idx):
        x = self.inputText[idx]
        outputText = self.outputText[idx]
        return x, outputText

dataset = CustomDataset(data["Fr"], data["En"])


In [ ]:
for x, y in dataset:
    print("x = ", x)
    print("y = ", y)
    break

x =  Va !
y =  Go.


In [ ]:
# Train, validation split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create a DataLoader to iterate over batches of data and performing preprocessing - By default produces batch first
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn = collate_fn, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, collate_fn = collate_fn)

In [ ]:
i = 0
for x, y in train_dataloader:
    print(x.shape)
    for src in x:
        print("X input", decode_sequence(src, SRC_LANGUAGE, vocab_transform[SRC_LANGUAGE]))
        # print("Y input", decode_sequence(tgt, TGT_LANGUAGE, vocab_transform[TGT_LANGUAGE]))
    print("y shape = ", y.shape)
    for item in y:
        print("Y labels", decode_sequence(item.T, TGT_LANGUAGE, vocab_transform[TGT_LANGUAGE]))
    if i == 1:
        break
    i += 1


torch.Size([64, 19])
X input  Tu as l' air superbe dans cette robe .
X input  Il n' y a aucun intérêt à rester ici .
X input  J' ai un rencard .
X input  Les lignes rouges sur la carte représentent les chemins de fer .
X input  Elle a toujours fait de son mieux pour leur rendre la vie plus facile .
X input  À quoi t' attendais - tu ?
X input  Je les déteste tous .
X input  Un changement de décor ne serait vraiment pas du luxe , pour moi !
X input  Elle veut me tuer .
X input  Eh bien , Tom , vous aviez raison .
X input  Il a pu réussir l' examen .
X input  On s' est débrouillé pour être là - bas à temps .
X input  Je ne veux pas être choisie .
X input  J' espère vraiment que vous allez apprécier le reste de votre journée .
X input  Il l' essaya encore et encore .
X input  C' est à peu près tout ce qu' il te faut savoir .
X input  J' ai du travail à faire , alors va - t' en et laisse -moi tranquille .
X input  Elle suggéra que je l' emmène au zoo .
X input  Tu es arrivé vite .
X input  

In [ ]:
# One head of self attention
class Head(nn.Module):
    def __init__(self, head_size, mask = True):
        super().__init__()
        # Query, key, and value are all linear layers.
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        # create a tril matrix of ones
        # PyTorch naming convention because the tril is not a parameter
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        self.mask = mask
    def forward(self, x, query = None, key = None, value = None):
        B,T,C = x.shape

        if key is None:
            key = x
        if query is None:
            query = x
        if value is None:
            value = x

        k = self.key(key)   # (B, T, head_size)
        q = self.query(query) # (B, T, head_size)
        v = self.value(value) # (B, T, head_size)

        # print("Head: x shape",x.shape) # (64, 16, 384)
        # print("Head: query shape", q.shape) # (64, 16, 64)
        # print("Head key shape", k.shape) # (64, 19, 64)
        # print("Head value shape", v.shape) # (64, 19, 64)
# 64 x 19 x 64
# 64 x 64 x 16
# w = 64 x 19 x 16
# 64 x 16 x 19
# v = 64 x 19 x 64
# Required = 64 x 16 x 64
        # Dot product the key and the query to get the weights
        w = k @ q.transpose(-2, -1)  # (B,T,H) @ (B,H,T) = (B, T, T)

        # Dividing by sqrt(head_size) for stability and making sure the variance stays close to zero
        w = w * (C ** -0.5)

        if self.mask:
            w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        w = F.softmax(w, dim = -1)
        w = self.dropout(w)

        out = w.transpose(-2, -1) @ v # (B, T, T) @ (B, T, C) = (B, T, C) cuz B stays the same so essentially its a (T, T) @ (T, C)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, no_of_heads, head_size, mask = True):
        super().__init__()
        self.mask = mask
        self.heads = nn.ModuleList([Head(head_size, self.mask) for _ in range(no_of_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, query = None, key = None, value = None):
        if key is None:
            key = x
        if query is None:
            query = x
        if value is None:
            value = x
        out = torch.cat([head(x, query, key, value) for head in self.heads], dim = -1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
        self.ln = nn.LayerNorm(n_embd)
    def forward(self, x):
            x = x + self.net(x)
            x = self.ln(x)
            return x

class GlobalSelfAttention(nn.Module):
    def __init__(self, n_embd, no_of_heads, mask = True):
        super().__init__()
        head_size = n_embd // no_of_heads
        self.mask = mask
        self.mha = MultiHeadAttention(no_of_heads, head_size, self.mask)
        self.ffwd = FeedForward(n_embd)
        self.ln = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.mha(x)
        x = self.ln(x)
        return x

class CrossAttention(nn.Module):
    def __init__(self, n_embd, no_of_heads):
        super().__init__()
        head_size = n_embd // no_of_heads
        self.ca = MultiHeadAttention(no_of_heads, head_size, mask = False)
        self.ln = nn.LayerNorm(n_embd)

    def forward(self, x, context):
        x = x + self.ca(x=x, query=x, key=context, value=context)
        x = self.ln(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, n_embd, no_of_heads):
        super().__init__()
        self.sa = GlobalSelfAttention(n_embd, no_of_heads, mask = False)
        self.ffn = FeedForward(n_embd)

    def forward(self, x):
        x = self.sa(x)
        x = self.ffn(x)
        return x
# class PositionalEncoding(nn.Module):
#     def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
#         super(PositionalEncoding, self).__init__()
#         den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
#         pos = torch.arange(0, maxlen).reshape(maxlen, 1)
#         pos_embedding = torch.zeros((maxlen, emb_size))
#         pos_embedding[:, 0::2] = torch.sin(pos * den)
#         pos_embedding[:, 1::2] = torch.cos(pos * den)
#         pos_embedding = pos_embedding.unsqueeze(-2)

#         self.dropout = nn.Dropout(dropout)
#         self.register_buffer('pos_embedding', pos_embedding)

#     def forward(self, token_embedding):
#         return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class Encoder(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[EncoderLayer(n_embd=n_embd, no_of_heads=no_of_heads) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets = None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        return x

class DecoderLayer(nn.Module):
    def __init__(self, n_embd, no_of_heads):
        super().__init__()
        self.masked_attn = GlobalSelfAttention(n_embd, no_of_heads, mask = True)
        self.crs_attn = CrossAttention(n_embd = n_embd, no_of_heads = no_of_heads)
        self.ffn = FeedForward(n_embd)

    def forward(self, x, context):
        x = self.masked_attn(x)
        # print("CA: Shape of x", x.shape)
        # print("CA: Shape of context", context.shape)
        x = self.crs_attn(x = x, context = context)
        # print("Decoder")
        x = self.ffn(x)
        return x


class Decoder(nn.Module):
    def __init__(self, vocab_size):
        super(Decoder, self).__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.dropout = nn.Dropout(dropout)
        self.blocks = nn.Sequential(*[DecoderLayer(n_embd=n_embd, no_of_heads=no_of_heads) for _ in range(n_layer)])

    def forward(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
        B, T = x.shape
        tok_emb = self.token_embedding_table(x)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb
        x = self.dropout(x)
        for i in range(n_layer):
            x  = self.blocks[i](x, context)
        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size_src, vocab_size_tgt, n_embd):
        super().__init__()
        self.encoder = Encoder(vocab_size_src)
        self.decoder = Decoder(vocab_size_tgt)
        self.final_layer = nn.Linear(n_embd, vocab_size_tgt)

    def forward(self, inputs):
        context, outputs = inputs
        context = context.to(device)
        outputs = outputs.to(device)
        y_input = outputs[:, :-1]
        y_labels = outputs[:, 1:]

        context = self.encoder(context)
        # print("output of context = ", context.shape)
        # print("Y input ", y_input.shape)
        x = self.decoder(x = y_input, context = context)  # (batch_size, target_len, d_model)
        logits = self.final_layer(x)

        return logits






# class BigramLanguageModel(nn.Module):
#     def __init__(self, vocab_size):
#         super().__init__()
#         self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
#         self.position_embedding_table = nn.Embedding(block_size, n_embd)
#         self.blocks = nn.Sequential(*[Block(n_embd, no_of_heads=no_of_heads) for _ in range(n_layer)])
#         self.ln_f = nn.LayerNorm(n_embd)
#         self.lm_head = nn.Linear(n_embd, vocab_size)

#     def forward(self, idx, targets = None):
#         B,T = idx.shape
#         tok_emb = self.token_embedding_table(idx) #(B,T,C)
#         pos_emb = self.position_embedding_table(torch.arange(T, device = device))
#         x = tok_emb + pos_emb
#         x = self.blocks(x)
#         x = self.ln_f(x)
#         logits = self.lm_head(x)
#         if targets is None:
#             loss = None
#         else:
#             # idx and targets are of shape (B,T)
#             B,T,C = logits.shape
#             logits = logits.view(B * T, C)
#             targets = targets.view(B * T)
#             loss = F.cross_entropy(logits, targets)
#         return logits, loss

#     def generate(self, idx, max_new_tokens):
#         #idx is (B,T)
#         for _ in range(max_new_tokens):
#             # Cropping the idx to the last block_size tokens
#             idx_cond = idx[:, -block_size:]
#             logits, loss = self(idx_cond)
#             logits = logits[:, -1, :] # Becomes (B, C)
#             probs = F.softmax(logits, dim = -1)

#             # Sampling from distribution
#             idx_next = torch.multinomial(probs, num_samples = 1)
#             idx = torch.cat((idx, idx_next), dim = 1)
#         return idx

# model = BigramLanguageModel(vocab_size)
# m = model.to(device)
# logits, loss = m(xb, yb)
# print(loss)

# idx = torch.zeros((1,1), dtype = torch.long, device = device) # stands for the new line token \n
# print(decode(m.generate(idx = idx, max_new_tokens = 100)[0].tolist()))


In [ ]:
# Unit test for encoder
sample_encoder = Encoder(vocab_size=vocab_size_src)
# x,y = next(iter(train_dataloader))
with torch.no_grad():
    sample_encoder_output = sample_encoder(x)

print("X shape = ", x.shape)
print(sample_encoder_output.shape)  # Shape `(B,T,C)`.

# Unit test for decoder
print(sample_encoder_output.shape)
sample_decoder = Decoder(vocab_size=vocab_size_tgt)

print("y.shape = ", y.shape)
with torch.no_grad():
    output = sample_decoder(
        x=y,
        context=torch.randn((y.shape[0], y.shape[1], n_embd)))

# Print the shapes.
print(y.shape)
print(output.shape)

X shape =  torch.Size([64, 19])
torch.Size([64, 19, 384])
torch.Size([64, 19, 384])
y.shape =  torch.Size([64, 17])
torch.Size([64, 17])
torch.Size([64, 17, 384])


In [ ]:
# Unit test for Transformer
transformer = Transformer(vocab_size_src = vocab_size_src, vocab_size_tgt = vocab_size_tgt, n_embd = n_embd)
output = transformer((x, y))

print(x.shape)
print(y.shape)
print(output.shape)

Head: x shape torch.Size([64, 19, 384])
Head: query shape torch.Size([64, 19, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 19])
Head: x shape torch.Size([64, 19, 384])
Head: query shape torch.Size([64, 19, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 19])
Head: x shape torch.Size([64, 19, 384])
Head: query shape torch.Size([64, 19, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 19])
Head: x shape torch.Size([64, 19, 384])
Head: query shape torch.Size([64, 19, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 19])
Head: x shape torch.Size([64, 19, 384])
Head: query shape torch.Size([64, 19, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 19])


In [ ]:
sample_ca = CrossAttention(n_embd, no_of_heads)
inp = torch.randn((64, 16, 384))
outp = torch.randn((64, 19, 384))
print(inp.shape)
print(outp.shape)
print(sample_ca(inp, outp).shape)

torch.Size([64, 16, 384])
torch.Size([64, 19, 384])
Head: x shape torch.Size([64, 16, 384])
Head: query shape torch.Size([64, 16, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 16])
Head: x shape torch.Size([64, 16, 384])
Head: query shape torch.Size([64, 16, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 16])
Head: x shape torch.Size([64, 16, 384])
Head: query shape torch.Size([64, 16, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 16])
Head: x shape torch.Size([64, 16, 384])
Head: query shape torch.Size([64, 16, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Size([64, 19, 64])
W shape = torch.Size([64, 19, 16])
Head: x shape torch.Size([64, 16, 384])
Head: query shape torch.Size([64, 16, 64])
Head key shape torch.Size([64, 19, 64])
Head value shape torch.Si